In [3]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as Fb
from torchtext.vocab import GloVe
from torchtext.legacy import data, datasets
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

a=4
%store a
%store

Stored 'a' (int)
Stored variables and their in-db values:
a             -> 4


In [4]:
class BIDAF(nn.Module):
    def __init__(self, pContext, pQuery,pVocabs):
        super(BIDAF, self).__init__()
        
        self.context = pContext
        self.query = pQuery
        self.vocabs = pVocabs
        #train set
        TEXT1 = data.Field(sequential=True, batch_first=True, lower=True)
        LABEL1 = data.Field(sequential=False, batch_first=True)
        TEXT1.build_vocab(self.vocabs, vectors=GloVe(name='6B', dim=100))
        LABEL1.build_vocab(self.vocabs)
        embedding_layer1 = nn.Embedding.from_pretrained(TEXT1.vocab.vectors, freeze=False)

        
        #charEmbedding Layer
        self.CharEmbedding = torch.nn.Sequential(
            nn.Conv1d(100, 100, (100,5)), #각기다른 필터가 각각의 출력값을 만들어내기를 희망
            torch.nn.Maxpool1d(Kernel_size=1))
        
        #Highway Layer
        self.t= nn.Sequential(Linear(hidden_size, hidden_size), nn.Sigmoid())  
        self.transformed = nn.Sequential(Linear(hidden_size, hidden_size),nn.ReLU())
        #그러네,, hiddensize는 dynamic하게 정해지는거 아닌가?, 어차피 shape을 아니까 그냥 내가 상수를 때려박아도 되는건가?
        
        #Contextual Layer
        self.context_lstm = nn.LSTM(input_size, hidden_size, bidirectional=True)
        
        #Attention Layer
        
        #Modeling Layer
        self.modeling_lstm= nn.LSTM(input_size, hidden_size, num_layers=2, bidirectional=True)
        
        #OutputLayer
        self.out_start = nn.Linear(input_size, num_of_tokens)
        self.out_end = nn.LSTM(input_size, num_of_tokens)
        self.softmax= nn.LogSoftmax(dim=1)
        
        
        
    #def forward(self, char1, glove1, char2, glove2):  #1은 context, 2는 query
    def forward(self, context, query):
        
        
        #input for context
        #char
        char_output1 = self.CharEmbedding(char1)
        
        #highway
        highway_input1 = torch.concat([char_output1, glove1]) #순서 상관있음??
        transformed_output1 = self.transforemd(highway_input1)
        t_ratio1 = self.t(highway_input1)
        highway_output1 = t_ratio1 * transformed_output1 + (1-t_ratio) * highway_input1
        
        
        #input for query
        #char
        char_output2 = self.CharEmbedding(char2)
        
        #highway
        highway_input2 = torch.concat([char_output2, glove2]) #순서 상관있음??
        transformed_output2 = self.transforemd(highway_input2)
        t_ratio2 = self.t(highway_input2)
        highway_output2 = t_ratio2 * transformed_output2 + (1-t_ratio) * highway_input2
        
        
        #contextual
        context_output, hidden = self.context_lstm(highway_output, hidden)
        LTR_order = context_output[:,0]
        RTL_order = context_output[:,1]
        context_output= torch.concat([LTR_order, RTL_order])
        
        #Attention
        
        attention_output = hidden #matrix g
        
        #modeling
        modeling_output, hidden = self.modeling_lstm(attention_output, hidden)
        LTR_order = modeling_output[:,0]
        RTL_order = modeling_output[:,1]
        modeling_output= torch.concat([LTR_order, RTL_order])
        
        #output
        dense = self.out_start(input)
        start= softmax(dense)
        #그냥 이렇게하는게 아니라 서로 concat하는 과정 추가해줘야함
        lstm, trash = out_end(start, num_of_tokens)
        end = softmax(lstm)
        
        return start, end
        
        
        

IndentationError: unexpected indent (1442255176.py, line 17)

In [6]:
#100 1D filters(kernels), each with a width of 5 ,,,,, glove dimension이 100이었으니까! 
#channel이 100개라고 생각해도 되는걸까? ,,,bias는 false?
class CharEmbedding(nn.Module):
    def __init__(self):
        super(CharEmbedding, self).__init__()
        self.p1 = torch.nn.Sequential(
            nn.Conv1d(100, 100, (100,5)), #각기다른 필터가 각각의 출력값을 만들어내기를 희망
            torch.nn.Maxpool1d(Kernel_size=1))
    
    def forward(self,x):
            out= self.p1(x)
            return out

    #누가봐도 이거아닌데 일단 넘어가자
    #정답이라면 out은 100d 여야함.

In [7]:
class ContextLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ContextLayer, self).__init__()
        self.input_size= input_size
        self.hidden_size =hidden_size
        self.context_lstm = nn.LSTM(input_size, hidden_size, bidirectional=True)
        #self.embedding = nn.Embedding(input_size, hidden_size) #이게 있어야 되나??,
        #아무리생각해도 embedding 없어도 될듯함. input자체에서 이미 embedding을 받고 오는데
        
    def forward(self, input, hidden):
        #embedded= self.embedding(input).view(1,1,-1)
        #output= embedded #([[forward1, backward1], [forward2, backward2]]) shape := [n_length, 2]
        output = input
        output, hidden = self.LSTM(output, hidden)
        LTR_order = output[:,0]
        RTL_order = output[:,1]
        return LTR_order, RTL_order #hidden은 굳이 return 할 필요가 없을 듯
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device=device) #꼭 0으로 초기화 해야되나?
    #  그리고 꼭 초기화를해야되나?? 없으면 자동 초기화인가?
        

In [4]:
class HighwayLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(HighwayLayer, self).__init__()
        self.input_size= input_size
        self.hidden_size =hidden_size
        
        self.t= nn.Sequential(Linear(hidden_size, hidden_size), nn.Sigmoid())
        self.transformed = nn.Sequential(Linear(hidden_size, hidden_size),nn.ReLU())

    def forward(self, glove, char):
        
        input = torch.concat([char, glove]) #순서 상관있음??
        transformed_input = transforemd(input)
        t_ratio = t(input)
        
        output = t_ratio * transformed_input + (1-t_ratio) * input
        return output
   
        

In [19]:
#신경망으로써는, 알파 fun 과 베타 fun의 linear layer인듯??  ,but beta function 은 자유인듯 
class AttentionLayer(nn.Module): 
    def __init__(self, input_size):
        super(AttentionLayer, self).__init__()
        self.input_size=  input_size
        self.alpha = nn.Linear(input_size, 1) #통과하면 scalar이고, 이걸통해서 similar matrix가되는데.
        #그렇다면 matrix로 받아서 한번에 S를 만들어야되는데 =>아닌듯
    def forward(self, input):
        out= self.alpha(input)
        return out
    
    #def initHidden(self):#이것도 있어야되나?
        
        
    
#layer는 class일뿐,,큰틀만 짓고 구체적인 코드는 python으로 구현.
#하지만 내부 attention다 클래스내부에서 구현을 해야함!

In [20]:
class ModelingLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(ModelingLayer, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.modeling_lstm= nn.LSTM(input_size, hidden_size, num_layers=2, bidirectional=True)
        
    def forward(self, input, hidden):
        output= input
        output, hidden = self.LSTM(output, hidden)
        LTR_order = output[:,0]
        RTL_order = output[:,1]
        return LTR_order, RTL_order 
        
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device=device) #꼭 0으로 초기화 해야되나?
    #  그리고 꼭 초기화를해야되나?? 없으면 자동 초기화인가?
            
        

In [21]:
#p1과 p2 학습 파라미터
class OutputLayer(nn.Module):
    def __init__(self, input_size, num_of_tokens): #num_of_tokens = context의 단어(토큰) 수
        super(ModelingLayer, self).__init__()
        self.input_size = input_size
        self.out_start = nn.Linear(input_size, num_of_tokens)
        self.out_end = nn.LSTM(input_size, num_of_tokens)
        self.softmax= nn.LogSoftmax(dim=1)
        
    def forward(self, input):#여기 input은 어디서 들어오는거지?
        dense = out_start(input)
        start= softmax(dense)
        lstm, trash=  out_end(start, num_of_tokens)
        end = softmax(lstm)
        
        return start, end
        
        
    
    